In [1]:
#calculo
import numpy as np
import pandas as pd

#grafico
import matplotlib.pyplot as plt
%matplotlib inline

#propios utilitarios
from utils import *

data_train = pd.read_csv('data/01dataBaseTrainTrxRec.csv', **set_parameter_csv)
data_train.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [2]:
#extra imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
import math as mt
import warnings

#extra config
matplotlib.style.use('bmh')
%matplotlib inline
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [3]:
data_train.sort_values(by=['codCliente','codEstab']).head(20)

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
25041,2017-05-24 00:00:00,1,105.0,39,1,147.0,1,0.084590
522851,2017-03-07 00:00:00,1,105.0,39,1,147.0,1,0.098481
743922,2017-08-20 00:00:00,1,105.0,39,1,147.0,1,0.018126
1272804,2017-05-13 00:00:00,1,105.0,39,1,147.0,1,0.018183
816568,2016-12-22 00:00:00,1,72.0,1194,1,147.0,1,0.031735
512607,2017-08-03 00:00:00,1,66.0,1911,1,136.0,1,0.119995
867506,2017-08-06 00:00:00,1,NaN,2399,0,NaN,1,0.009543
1331328,2016-11-20 00:00:00,1,110.0,2425,1,170.0,1,0.012931
1478724,2017-06-20 00:00:00,1,110.0,2425,1,170.0,1,0.011858
400101,2017-08-26 00:00:00,1,50.0,3843,1,170.0,1,0.056412


In [4]:
data_perfil = pd.read_csv('data/02dataBasePerfilRec.csv')
data_perfil.head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,24041,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
1,9040,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN


In [5]:
null_verificator(data_perfil)

,Nulos,Cantidad,Tipo Col
codCliente,False,0,int64
rangoEdad,True,57,object
rangoIngreso,True,1263,object
flagGenero,False,0,int64
flagLimaProvCliente,False,0,int64
ubigeoCliente,True,4,float64
rangoCtdProdAct,False,0,object
rangoCtdProdPas,False,0,object
rangoCtdProdSeg,False,0,object
flagBxi,False,0,int64
